### Modelo de Recomendacion ###

Se diseñó e implementó un sistema de recomendación basado en el modelo item-item. Tras el pre-procesamiento de datos de juegos, se calculó una matriz de similitud de coseno para establecer relaciones entre los ítems. A partir de esta matriz, se desarrolló una función de recomendación que, dada la identificación de un juego, devuelve una lista de cinco juegos similares. Este sistema ha sido integrado en una API para proporcionar recomendaciones personalizadas a los usuarios finales."

Importamos las librerias necesarias para llavar a cabo el proceso

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
df_genres = pd.read_parquet("df_herramientas_dummies.parquet")
df_games = pd.read_parquet("output_steam_games_limpio.parquet")

In [3]:
# fusionamos df_games y df_genres en df_fusion usando la columna 'id' 
df_fusion = df_games.merge(df_genres, on='id', how='left')

#Creamos la lista caracteristicas que incluye 'release_date' y todas las columnas de df_genres excepto la primera
caracteristicas = ['release_date'] + list(df_genres.columns[1:])  

# Se crea un objeto para normalizar'release_date' en df_fusion, escalando los valores para que tengan una media de 0 y una desviación estándar de 1.
escala = StandardScaler()
df_fusion['release_date'] = escala.fit_transform(df_fusion[['release_date']])

df_final = df_fusion[['id'] + caracteristicas]
df_final= df_final.merge(df_games[['id', 'app_name']], on='id', how='left')

In [4]:
# Hacemos un sample de un subconjunto de los datos (20%)
df_sampled = df_final.sample(frac=0.2, random_state=42)

# Calculamos similitud del coseno 
matriz_similitud = cosine_similarity(df_sampled[caracteristicas].fillna(0))
matriz_similitud = np.nan_to_num(matriz_similitud)

In [5]:
def juego_recomendado(game_id, top_n=5):
    ids_juegos_muestra = df_sampled['id'].unique() # obtenemos ids de juegos
    
    # Verificamos si el ID del juego está en los juegos muestreados
    if game_id not in ids_juegos_muestra:
        return f"No se encontraron recomendaciones,  {game_id} no está en los datos muestreados."
    
    # Obtenemos el índice del juego en los datos
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    # Verificamos si se encontró el juego en los datos muestreados
    if not indice_juego:
        return f"No se encontraron recomendaciones,  {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]

    # Calculamos los puntajes de similitud entre juegos
    puntajes = list(enumerate(matriz_similitud[indice_juego]))
    puntajes = sorted(puntajes, key=lambda x: x[1], reverse=True)

    # Obtenemos los índices de los juegos similares
    indices_juegos_similares = [i for i, puntaje in puntajes[1:top_n+1]]
    
    # Obtenemos los nombres de los juegos similares
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

    # Mensaje de recomendación
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego: {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares

In [6]:
test_game_id = "774276"
recommendations = juego_recomendado(test_game_id, top_n=5)

print("Recomendaciones para ID de juego :", test_game_id)
for rec in recommendations:
    print(rec)

Recomendaciones para ID de juego : 774276
Juegos recomendados basados en el ID del juego: 774276 - Imhotep, Pyramid Builder:
TS Marketplace: Feather River Canyon Scenario Pack 01
FSX: Steam Edition - Grumman HU-16B Albatross Add-On
TANE DLC: Newcastle Shunter
FSX Steam Edition: Endelave Airport Add-On
FSX: Steam Edition - Nordborg Airfield Add-On
